In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights="imagenet",
                                include_top=False,
                  ###TODO_学習データの画像サイズが224,224以外の場合は変更してください####
                                input_shape=(224,224,3))
conv_base.trainable = False

#全結合分類機を合わせる
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, kernel_regularizer=regularizers.l2(0.001), activation="relu"))
model.add(layers.Dense(2,activation="softmax"))#プリンorカービィなので2にする。
model.summary()

#モデルのコンパイル
model.compile(loss="categorical_crossentropy",
             optimizer=optimizers.RMSprop(lr=1e-5),
             metrics=["acc"])

In [0]:
#データの準備
from keras.utils import np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os

categories = ["プリン", "カービィ"]
nb_classes = len(categories)

###TODO_numpy_converterで出力したファイル名をinput_numpyに設定###
input_numpy = "sample.npy"
X_train, X_test, Y_train, Y_test = np.load("/content/drive/My Drive/Colab Notebooks/"+input_numpy, allow_pickle=True)

#データの正規化
X_train = X_train.astype("float") / 255
X_test = X_test.astype("float") / 255

#kerasで扱えるようにcategoriesをベクトルに変換
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

#ModelCheckpointを設定する
modelCheckpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/Colab Notebooks/model/transfer_learning/model_{epoch:02d}.h5', verbose=1, period=10)

#ModelCheckpointで出力されるモデルの保存先フォルダを作成する
try:
  os.makedirs("/content/drive/My Drive/Colab Notebooks/model/transfer_learning/")
except FileExistsError:
  pass

#学習を実行
model = model.fit(X_train, Y_train, epochs=200, batch_size=64, validation_data=(X_test, Y_test), callbacks=[modelCheckpoint])


In [0]:
#学習結果をグラフで表示する。
import matplotlib.pyplot as plt

acc = model.history['acc']
val_acc = model.history['val_acc']
loss = model.history['loss']
val_loss = model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

# 新しいセクション